In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.stats import percentileofscore
from matplotlib.gridspec import GridSpec
import matplotlib.colors as mcolors
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
from matplotlib.lines import Line2D
import matplotlib as mpl
from matplotlib import font_manager
mpl.rcParams.update(mpl.rcParamsDefault)

# Define the scope and load credentials
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials_path = "C:/Users/benoi/OneDrive/Desktop/bea/json credentials/bea-data-7dda3770b44f.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)

# Authenticate and connect to Google Sheets
client = gspread.authorize(creds)
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1OfgrNbgZjwMMAuVdIhOLywXUHEw1xHzYaN8ZU2awsSs/edit?gid=1169755047')
worksheet = sheet.get_worksheet(0)
data = pd.DataFrame(worksheet.get_all_records())

metric_dict = {
    # Pitching metrics
    ('Trackman Bullpen', 'Fastball'): ['Max Velo', 'Average Velo'],
    ('Roll Ins', '5oz'): ['Max Velo', 'Average Velo'],
    ('Double Plays', '5oz'): ['Max Velo', 'Average Velo'],
    ('Turn and Burns', '5oz'): ['Max Velo', 'Average Velo'],
    ('Pulldowns', '5oz'): ['Max Velo', 'Average Velo'],
    ('Roll Ins', '7oz'): ['Max Velo', 'Average Velo'],
    ('Double Plays', '7oz'): ['Max Velo', 'Average Velo'],
    ('Turn and Burns', '7oz'): ['Max Velo', 'Average Velo'],
    ('Pulldowns', '7oz'): ['Max Velo', 'Average Velo'],
    ('Roll Ins', '6oz'): ['Max Velo', 'Average Velo'],
    ('Double Plays', '6oz'): ['Max Velo', 'Average Velo'],
    ('Turn and Burns', '6oz'): ['Max Velo', 'Average Velo'],
    ('Pulldowns', '6oz'): ['Max Velo', 'Average Velo'],
    ('Roll Ins', '3oz'): ['Max Velo', 'Average Velo'],
    ('Double Plays', '3oz'): ['Max Velo', 'Average Velo'],
    ('Turn and Burns', '3oz'): ['Max Velo', 'Average Velo'],
    ('Pulldowns', '3oz'): ['Max Velo', 'Average Velo'],
    ('Roll Ins', '4oz'): ['Max Velo', 'Average Velo'],
    ('Double Plays', '4oz'): ['Max Velo', 'Average Velo'],
    ('Turn and Burns', '4oz'): ['Max Velo', 'Average Velo'],
    ('Pulldowns', '4oz'): ['Max Velo', 'Average Velo'],
    ('Mound Velo', '5oz'): ['Max Velo', 'Average Velo'],
    ('Catchers Velo', '5oz'): ['Max Velo', 'Average Velo'],
    ('Catchers Velo', '6oz'): ['Max Velo', 'Average Velo'],
    ('Catchers Velo', '7oz'): ['Max Velo', 'Average Velo'],
    ('Catchers Velo', '4oz'): ['Max Velo', 'Average Velo'],
    ('Catchers Velo', '3oz'): ['Max Velo', 'Average Velo'],
    # Jump metrics
    ('Jump', 'Broad'): ['Distance'],
    ('Jump', 'Vertical'): ['Vert'],
    #Grip metrics
    ('Grip', 'Arm Side'): ['Weight'],
    ('Grip', 'Glove Side'): ['Weight'],
    # Strength metrics
    ('Back Squat', 'Starting Strength'): ['Weight'],
    ('Deadlift', 'Starting Strength'): ['Weight'],
    ('Bench Press', 'Starting Strength'): ['Weight'],
    ('Back Squat', 'Strength-Speed'): ['Weight'],
    ('Deadlift', 'Strength-Speed'): ['Weight'],
    ('Bench Press', 'Strength-Speed'): ['Weight'],
    ('Back Squat', 'Accelerative Strength'): ['Weight'],
    ('Deadlift', 'Accelerative Strength'): ['Weight'],
    ('Bench Press', 'Accelerative Strength'): ['Weight'],
    ('Back Squat', 'Absolute Strength'): ['Weight'],
    ('Deadlift', 'Absolute Strength'): ['Weight'],
    ('Bench Press', 'Absolute Strength'): ['Weight'],
    # Hitting metrics
    ('HitTrax', 'Front Toss'): ['Average EV', 'Max EV', 'Max Distance'],
    ('HitTrax', 'Tee'): ['Average EV', 'Max EV', 'Max Distance'],
    ('HitTrax', 'Machine/BP'): ['Average EV', 'Max EV', 'Max Distance'],
    ('HitTrax', 'Live AB'): ['Average EV', 'Max EV', 'Max Distance'],
    ('Blast Motion', 'Front Toss'): ['Max Bat Speed', 'Peak Hand Speed'],
    ('Blast Motion', 'Tee'): ['Max Bat Speed', 'Peak Hand Speed'],
    ('Blast Motion', 'Machine/BP'): ['Max Bat Speed', 'Peak Hand Speed'],
    ('Blast Motion', 'Live AB'): ['Max Bat Speed', 'Peak Hand Speed'],
    # ArmCare metric
    ('ArmCare', 'Fresh Exam'): ['Arm Score'],
    #Weigh-in
    ('Weigh-in', 'Weigh-in'): ['Weight'],
    ('Weigh-in', 'Height'): ['Height']
}

unit_dict = {
    ('Trackman Bullpen', 'Fastball'): 'MPH',
    ('Roll Ins', '5oz'): 'MPH',
    ('Double Plays', '5oz'): 'MPH',
    ('Turn and Burns', '5oz'): 'MPH',
    ('Pulldowns', '5oz'): 'MPH',
    ('Roll Ins', '7oz'): 'MPH',
    ('Double Plays', '7oz'): 'MPH',
    ('Turn and Burns', '7oz'): 'MPH',
    ('Pulldowns', '7oz'): 'MPH',
    ('Roll Ins', '6oz'): 'MPH',
    ('Double Plays', '6oz'): 'MPH',
    ('Turn and Burns', '6oz'): 'MPH',
    ('Pulldowns', '6oz'): 'MPH',
    ('Roll Ins', '3oz'): 'MPH',
    ('Double Plays', '3oz'): 'MPH',
    ('Turn and Burns', '3oz'): 'MPH',
    ('Pulldowns', '3oz'): 'MPH',
    ('Roll Ins', '4oz'): 'MPH',
    ('Double Plays', '4oz'): 'MPH',
    ('Turn and Burns', '4oz'): 'MPH',
    ('Pulldowns', '4oz'): 'MPH',
    ('Jump', 'Broad'): 'feet',
    ('Jump', 'Vertical'): 'inches',
    ('Grip', 'Arm Side'): 'lbs',
    ('Grip', 'Glove Side'): 'lbs',
    ('Back Squat', 'Starting Strength'): 'lbs',
    ('Deadlift', 'Starting Strength'): 'lbs',
    ('Bench Press', 'Starting Strength'): 'lbs',
    ('Back Squat', 'Strength-Speed'): 'lbs',
    ('Deadlift', 'Strength-Speed'): 'lbs',
    ('Bench Press', 'Strength-Speed'): 'lbs',
    ('Back Squat', 'Accelerative Strength'): 'lbs',
    ('Deadlift', 'Accelerative Strength'): 'lbs',
    ('Bench Press', 'Accelerative Strength'): 'lbs',
    ('Back Squat', 'Absolute Strength'): 'lbs',
    ('Deadlift', 'Absolute Strength'): 'lbs',
    ('Bench Press', 'Absolute Strength'): 'lbs',
    ('ArmCare', 'Fresh Exam'): '',
    ('Weigh-in', 'Weigh-in'): 'lbs',
    ('Weigh-in', 'Height'): 'inches',
    ('Blast Motion', 'Front Toss', 'Max Bat Speed'): 'MPH',
    ('Blast Motion', 'Front Toss', 'Peak Hand Speed'): 'MPH',
    ('HitTrax', 'Front Toss', 'Average EV'): 'MPH',
    ('HitTrax', 'Front Toss', 'Max EV'): 'MPH',
    ('HitTrax', 'Front Toss', 'Max Distance'): 'ft',
    ('Blast Motion', 'Tee', 'Max Bat Speed'): 'MPH',
    ('Blast Motion', 'Tee', 'Peak Hand Speed'): 'MPH',
    ('HitTrax', 'Tee', 'Average EV'): 'MPH',
    ('HitTrax', 'Tee', 'Max EV'): 'MPH',
    ('HitTrax', 'Tee', 'Max Distance'): 'ft',
    ('Blast Motion', 'Machine/BP', 'Max Bat Speed'): 'MPH',
    ('Blast Motion', 'Machine/BP', 'Peak Hand Speed'): 'MPH',
    ('HitTrax', 'Machine/BP', 'Average EV'): 'MPH',
    ('HitTrax', 'Machine/BP', 'Max EV'): 'MPH',
    ('HitTrax', 'Machine/BP', 'Max Distance'): 'ft',
    ('Blast Motion', 'Live AB', 'Max Bat Speed'): 'MPH',
    ('Blast Motion', 'Live AB', 'Peak Hand Speed'): 'MPH',
    ('HitTrax', 'Live AB', 'Average EV'): 'MPH',
    ('HitTrax', 'Live AB', 'Max EV'): 'MPH',
    ('HitTrax', 'Live AB', 'Max Distance'): 'ft'
}

def calculate_percentiles(df, athlete_id, metric_dict, selected_metrics=None, verbose=False):
  
    # Initialize percentiles and athlete details
    percentiles = {}
    athlete_data = df[df['ID'] == athlete_id]

    # Check if athlete data exists
    if athlete_data.empty:
        if verbose:
            print(f"No data found for athlete ID {athlete_id}.")
        return {}, None, None

    # Get athlete's name and current level
    athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"
    athlete_level = athlete_data['Level'].iloc[-1]  # Use the most recent level

    # Filter data for the athlete's current level
    peer_data = df[df['Level'] == athlete_level]
    if peer_data.empty:
        if verbose:
            print(f"No peer data found for level '{athlete_level}'.")
        return {}, athlete_name, athlete_level

    if verbose:
        print(f"Processing data for {athlete_name} at level '{athlete_level}' with {len(peer_data)} peers.")

    # Process metrics
    for (test_type, sub_type), metrics in metric_dict.items():
        for metric in metrics:
            # Skip if not in selected metrics
            if selected_metrics and metric not in selected_metrics:
                continue

            subset = peer_data[(peer_data['Test Type'] == test_type) &
                               (peer_data['Test Sub-Type'] == sub_type)]

            if metric not in subset.columns:
                if verbose:
                    print(f"Metric '{metric}' not found in data for {test_type} - {sub_type}")
                continue

            # Ensure `subset` is a copy before modifying
            subset = subset.copy()
            
            # Convert to numeric and drop NaN
            subset[metric] = pd.to_numeric(subset[metric], errors='coerce')
            subset = subset.dropna(subset=[metric])

            # Skip if subset is empty
            if subset.empty:
                if verbose:
                    print(f"No valid data for metric '{metric}' under {test_type} - {sub_type}.")
                continue

            athlete_subset = subset[subset['ID'] == athlete_id].sort_values(by='Test Number')

            # Handle cases with fewer than two tests
            if athlete_subset.empty:
                if verbose:
                    print(f"No test data available for metric '{metric}' for athlete ID {athlete_id}.")
                continue

            # Extract test values
            first_test = athlete_subset.iloc[0][metric]  # First test value
            most_recent_test = athlete_subset.iloc[-1][metric]  # Most recent test value
            previous_test = athlete_subset.iloc[-2][metric] if len(athlete_subset) > 1 else first_test

            # Ensure no NaN values are included in calculations
            first_test = first_test if pd.notna(first_test) else 0
            most_recent_test = most_recent_test if pd.notna(most_recent_test) else 0
            previous_test = previous_test if pd.notna(previous_test) else 0

            # Calculate percentile ranks
            first_percentile = percentileofscore(subset[metric], first_test)
            recent_percentile = percentileofscore(subset[metric], most_recent_test)
            previous_percentile = percentileofscore(subset[metric], previous_test)

            # Store results
            label = f"{test_type} - {sub_type} - {metric}"
            percentiles[label] = {
                'first_percentile': first_percentile,
                'previous_percentile': previous_percentile,
                'recent_percentile': recent_percentile,
                'first_value': first_test,
                'previous_value': previous_test,
                'recent_value': most_recent_test
            }

            if verbose:
                print(f"Calculated percentiles for {label}: {percentiles[label]}")

    return percentiles, athlete_name, athlete_level

# Radar Chart Function for Specific Metrics
def radar_chart_percentiles(df, percentiles, athlete_name, athlete_id, pdf, unit_dict, verbose=False):
   
    # Retrieve the athlete's current level
    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        print(f"No data found for athlete ID {athlete_id}.")
        return

    athlete_level = athlete_data['Level'].iloc[-1]

    # Filter data for the athlete's current level
    level_data = athlete_data[athlete_data['Level'] == athlete_level]
    if level_data.empty:
        print(f"No data available for athlete at level '{athlete_level}'.")
        return

    # Define metrics for the radar chart
    selected_metrics = [
        ('Weigh-in', 'Weigh-in', 'Weight'),
        ('Weigh-in', 'Height', 'Height'),
        ('Trackman Bullpen', 'Fastball', 'Max Velo'),
        ('Trackman Bullpen', 'Fastball', 'Average Velo'),
        ('Blast Motion', 'Front Toss', 'Max Bat Speed'),
        ('Blast Motion', 'Front Toss', 'Peak Hand Speed'),
        ('HitTrax', 'Front Toss', 'Average EV'),
        ('HitTrax', 'Front Toss', 'Max EV'),
        ('HitTrax', 'Front Toss', 'Max Distance'),
        ('Roll Ins', '5oz', 'Max Velo'),
        ('Double Plays', '5oz', 'Max Velo'),
        ('Turn and Burns', '5oz', 'Max Velo'),
        ('Pulldowns', '5oz', 'Max Velo'),
        ('Jump', 'Broad', 'Distance'),
        ('Jump', 'Vertical', 'Vert'),
        ('Back Squat', 'Strength-Speed', 'Weight'),
        ('Deadlift', 'Strength-Speed', 'Weight'),
        ('Bench Press', 'Strength-Speed', 'Weight'),
        ('ArmCare', 'Fresh Exam', 'Arm Score'),
    ]

    # Define custom labels for the metrics
    custom_labels = [
        "Body Weight",
        "Height",
        "Fastball Max Velocity",
        "Fastball Average Velocity",
        "Max Bat Speed",
        "Peak Hand Speed",
        "Average Exit Velo",
        "Max Exit Velo",
        "Max Distance",
        "5oz Roll Ins",
        "5oz Double Plays",
        "5oz Turn and Burns",
        "5oz Pulldown",
        "Broad Jump Distance",
        "Vertical Jump Height",
        "Speed Back Squat",
        "Speed Deadlift",
        "Speed Bench Press",
        "ArmCare Score",
    ]

    # Initialize values for plotting
    labels = []
    onboarding_values = []
    recent_values = []
    onboarding_actuals = []
    recent_actuals = []
    onboarding_dates = []

    for i, (test_type, sub_type, metric) in enumerate(selected_metrics):
        metric_data = level_data[
            (level_data['Test Type'] == test_type) & (level_data['Test Sub-Type'] == sub_type)
        ]
        if metric_data.empty:
            labels.append(f"{custom_labels[i]}\n(N/A)")
            onboarding_values.append(0)
            recent_values.append(0)
            onboarding_actuals.append(None)
            recent_actuals.append(None)
            onboarding_dates.append(None)
            continue

        # Extract most recent test by max Test Number
        most_recent_test = metric_data.loc[metric_data['Test Number'].idxmax()]
        onboarding_test = metric_data.loc[metric_data['Test Number'].idxmin()]

        # Extract the earliest test date for onboarding
        onboarding_date = pd.to_datetime(onboarding_test['Date']).strftime('%m/%d/%Y')
        onboarding_dates.append(onboarding_date)

        # Validate the extracted test
        if verbose:
            print(f"Metric: {custom_labels[i]}")
            print(f"Onboarding Test: {onboarding_test[['Test Number', 'Date']].to_dict()}")
            print(f"Most Recent Test: {most_recent_test[['Test Number', 'Date']].to_dict()}\n")

        # Extract date for the label
        latest_test_date = pd.to_datetime(most_recent_test['Date']).strftime('%m/%d/%y')
        labels.append(f"{custom_labels[i]}\n({latest_test_date})")

        # Extract percentiles and actual values
        metric_key = f"{test_type} - {sub_type} - {metric}"
        onboarding_values.append(
            percentiles[metric_key]['first_percentile'] if metric_key in percentiles else 0
        )
        recent_values.append(
            percentiles[metric_key]['recent_percentile'] if metric_key in percentiles else 0
        )
        onboarding_actuals.append(
            percentiles[metric_key]['first_value'] if metric_key in percentiles else None
        )
        recent_actuals.append(
            percentiles[metric_key]['recent_value'] if metric_key in percentiles else None
        )

    # Ensure the radar chart is circular
    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
    onboarding_values += onboarding_values[:1]
    recent_values += recent_values[:1]
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(14, 12), subplot_kw=dict(polar=True))

    # Debugging: Check the list of onboarding dates before taking the minimum
    valid_onboarding_dates = [date for date in onboarding_dates if date is not None]

    if valid_onboarding_dates:
        legend_onboarding_date = min(valid_onboarding_dates)
    else:
        legend_onboarding_date = "N/A"

    # Print statement for debugging
    if verbose:
        print(f"Legend Onboarding Date: {legend_onboarding_date}")

    try:
    # Plot onboarding data
        ax.plot(angles, onboarding_values, color='blue', linewidth=1, linestyle='dotted', 
                label=f'Onboarding ({legend_onboarding_date})')
        ax.fill(angles, onboarding_values, color='blue', alpha=0.1)

        # Plot most recent data
        ax.plot(angles, recent_values, color='red', linewidth=1, linestyle='solid', label='Most Recent')
        ax.fill(angles, recent_values, color='red', alpha=0.1)

    # Annotate actual values with units
        for angle, onboarding_value, recent_value, metric_label in zip(angles, onboarding_actuals, recent_actuals, selected_metrics):
            unit = unit_dict.get((metric_label[0], metric_label[1]), '')


            # Determine positions based on greater and lesser value
            if onboarding_value is not None and recent_value is not None:
                if onboarding_value > recent_value:
                    outer_value, inner_value = onboarding_value, recent_value
                    outer_color, inner_color = 'blue', 'red'
                else:
                    outer_value, inner_value = recent_value, onboarding_value
                    outer_color, inner_color = 'red', 'blue'


                ax.text(angle, 95, f"{outer_value:.1f} {unit}", ha='center', fontsize=8, color=outer_color)
                ax.text(angle, 75, f"{inner_value:.1f} {unit}", ha='center', fontsize=8, color=inner_color)
            elif onboarding_value is not None:
                ax.text(angle, 80, f"{onboarding_value:.1f} {unit}", ha='center', fontsize=8, color='blue')
            elif recent_value is not None:
                ax.text(angle, 80, f"{recent_value:.1f} {unit}", ha='center', fontsize=8, color='red')

         # Set fixed radial limits and gridlines
        ax.set_ylim(0, 105)  # Ensure values stay within the 0-100 range
        ax.set_yticks([0, 20, 40, 60, 80, 100])  # Fixed increments
        ax.set_yticklabels(['0', '20', '40', '60', '80', '100'], fontsize=10)

        # Finalize the radar chart
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(labels, fontsize=10)
        ax.set_title(
            f"{athlete_name} - Growth Chart\nLevel: {athlete_level}",
            fontsize=14,
            fontweight='bold'
        )
        ax.legend(loc='upper right')
        plt.tight_layout()

        # Debugging: Confirm before saving
        if verbose:
            print("Saving radar chart to PDF...")

        # Save the figure to the PDF
        pdf.savefig(fig)
        plt.close(fig)

        # Debugging: Confirm success
        if verbose:
            print("Radar chart successfully saved.")
    
    except Exception as e:
        print(f"Error generating radar chart: {e}")

def generate_metric_table(df, athlete_id, selected_metrics, unit_dict, pdf, custom_labels):
    
    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        print(f"No data found for athlete ID {athlete_id}.")
        return

    # Initialize table data
    table_data = []
    column_headers = ["Metric", "Onboarding", "Most Recent", "PR"]

    for idx, (test_type, test_sub_type, metric) in enumerate(selected_metrics):
        metric_label = custom_labels[idx]
        metric_data = athlete_data[
            (athlete_data['Test Type'] == test_type) & (athlete_data['Test Sub-Type'] == test_sub_type)
        ]

        if metric_data.empty:
            onboarding_value = most_recent_value = pr_value = "N/A"
            onboarding_info = most_recent_info = pr_info = "N/A"
        else:
            if metric in metric_data.columns:
                metric_data = metric_data.copy()
                metric_data[metric] = pd.to_numeric(metric_data[metric], errors='coerce')

            def extract_value_info(data):
                """ Extract metric value and corresponding date/phase info """
                if not data.empty and metric in data.columns:
                    value = data[metric].iloc[0]

                    # Ensure the 'Date' column is properly formatted as datetime
                    if 'Date' in data.columns:
                        data = data.copy()
                        data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

                    date_phase = f"{data['Date'].iloc[0].strftime('%m/%d/%y')}" if pd.notna(data['Date'].iloc[0]) else "N/A"

                    if 'Phase' in data.columns and pd.notna(data['Phase'].iloc[0]):
                        date_phase += f", {data['Phase'].iloc[0]}"

                    return value, date_phase

                return "N/A", "N/A"

            # Extract values for Onboarding, Most Recent, PR
            onboarding_value, onboarding_info = extract_value_info(metric_data[metric_data['Test Number'] == 1])
            most_recent_value, most_recent_info = extract_value_info(
                metric_data[metric_data['Test Number'] == metric_data['Test Number'].max()]
            )
            pr_value = metric_data[metric].max() if metric in metric_data.columns else "N/A"
            pr_info = extract_value_info(metric_data[metric_data[metric] == pr_value])[1] if pr_value != "N/A" else "N/A"

        # Get units for the metric
        units = unit_dict.get((test_type, test_sub_type), "")

        def format_with_units(value):
            return f"{value} {units}" if value != "N/A" and units else value

        # Format values
        onboarding_value = format_with_units(onboarding_value)
        most_recent_value = format_with_units(most_recent_value)
        pr_value = format_with_units(pr_value)

        def format_bold_metric(value, date_phase):
            """ Bold the metric value but keep date/phase normal. """
            return f"$\\bf{{{value}}}$\n{date_phase}" if value != "N/A" else "N/A"

        # **Append the formatted row to the table_data**
        table_data.append([
            metric_label,  # Metric name (already fixed for spacing)
            format_bold_metric(onboarding_value, onboarding_info),
            format_bold_metric(most_recent_value, most_recent_info),
            format_bold_metric(pr_value, pr_info),
        ])

    # **Create the table plot (only once, after loop finishes)**
    fig, ax = plt.subplots(figsize=(12, len(table_data) * 0.6 + 2))  # Adjust height dynamically
    ax.axis("off")  # Remove axes

    # **Now, create the table with finalized `table_data`**
    table = ax.table(
        cellText=table_data,
        colLabels=column_headers,
        loc="center",
        cellLoc="center",
        colColours=["#4f81bd"] * len(column_headers),  # Blue header background
        bbox=[0, 0, 1, 1],  # Cover the entire figure
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.auto_set_column_width(col=list(range(len(column_headers))))

    # Format headers
    for (row, col), cell in table.get_celld().items():
        if row == 0:  # Header row
            cell.set_text_props(weight="bold", color="white", fontsize=9)

    # **Bold only the first column manually (fix metric name bolding)**
    for row in range(1, len(table_data) + 1):
        table.get_celld()[(row, 0)].set_text_props(weight="bold")  # Bold metric name

    # Alternating row colors for readability
    for row, col in table.get_celld():
        if row > 0:  # Skip header row
            if row % 2 == 0:
                table.get_celld()[(row, col)].set_facecolor("#f2f2f2")  # Light gray
            else:
                table.get_celld()[(row, col)].set_facecolor("white")  # White

    # Add title
    ax.set_title(
        "Metric Summary",
        fontsize=16,
        fontweight="bold",
        loc="center",
        pad=20,
    )

    # Save the table to the PDF
    pdf.savefig(fig)
    plt.close(fig)

def bar_chart_improvement(df, athlete_id, metric_dict, unit_dict, pdf):
   
    # Retrieve percentiles and athlete information based on current level
    percentiles, athlete_name, athlete_level = calculate_percentiles(df, athlete_id, metric_dict)

    # Create metrics summary and calculate improvement
    metrics_summary = pd.DataFrame(percentiles).T
    metrics_summary['Improvement'] = metrics_summary['recent_percentile'] - metrics_summary['previous_percentile']

    # Sort and reverse the order to have most improved at the top
    combined_metrics = metrics_summary.sort_values(by='Improvement', ascending=False).iloc[::-1]

    # Generate colors for the bars
    cmap = plt.colormaps.get_cmap('coolwarm')
    colors = cmap(np.linspace(0, 1, len(combined_metrics)))

    fig, ax = plt.subplots(figsize=(14, 10))

    for i, (metric, row) in enumerate(combined_metrics.iterrows()):
        # Get units and define color
        metric_parts = metric.split(' - ')
        units = unit_dict.get(tuple(metric_parts[:2]), '')  # Extract test type and sub-type for unit lookup
        color = colors[i]

        # Plot bars for recent and previous percentiles
        ax.barh(metric, row['recent_percentile'], color=color, alpha=0.7)
        ax.barh(metric, row['previous_percentile'], color='lightgray', alpha=0.5)

        # Add metric value annotation
        if pd.notna(row['recent_value']):
            ax.text(row['recent_percentile'] + 1, i, f"{row['recent_value']:.1f} {units}",
                    va='center', ha='left', fontsize=8, color='black')

        # Position the improvement annotation conditionally based on positive or negative improvement
        improvement_text = f"{row['Improvement']:+.1f}"
        if row['Improvement'] < 0:
            # Position to the left of the bar for negative improvement
            ax.text(row['previous_percentile'] - 1, i, improvement_text, va='center', ha='right',
                    fontsize=8, color='white')
        else:
            # Centered over the bar for positive improvement
            ax.text((row['previous_percentile'] + row['recent_percentile']) / 2, i,
                    improvement_text, va='center', ha='center', fontsize=8, color='white')

    # Finalize plot details
    ax.set_xlim(0, 100)
    ax.set_xlabel("Percentile Rank")
    ax.set_title(f"Most and Least Improved Metrics\nLevel: {athlete_level}", fontweight='bold')
    ax.grid(axis='x', color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
    plt.tight_layout()

    # Save the figure to the PDF
    pdf.savefig(fig)
    plt.close(fig)

def generate_strength_radar_chart(df, athlete_id, pdf):
    athlete_data = df[(df['ID'] == athlete_id) & (df['Workout Type'] == 'Strength')]

    if athlete_data.empty:
        print("No strength test data found for this athlete.")
        return

    test_types = ['Back Squat', 'Bench Press', 'Deadlift']
    sub_types = ['Starting Strength', 'Strength-Speed', 'Accelerative Strength', 'Absolute Strength']

    fig = plt.figure(figsize=(14, 12))
    gs = GridSpec(3, 2, height_ratios=[3, 3, 3], hspace=0.3, wspace=0)

    athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"

    # Set up a grouped legend
    legend_elements = [
        Line2D([0], [0], color='none', label=''),
        Line2D([0], [0], color='r', lw=5, label='Recent VBT Test'),
        Line2D([0], [0], color='b', lw=5, label='Previous VBT Test'),
        Line2D([0], [0], color='none', label=''),
        Line2D([0], [0], color='none', marker='', label='Starting Strength = 1.0-1.3 m/s'),
        Line2D([0], [0], color='none', marker='', label='Strength-Speed = 0.8-0.9 m/s'),
        Line2D([0], [0], color='none', marker='', label='Accelerative Strength = 0.5-0.7 m/s'),
        Line2D([0], [0], color='none', marker='', label='Absolute Strength = <0.4 m/s')
    ]

    for i, test_type in enumerate(test_types):
        ax = fig.add_subplot(gs[i, 0], polar=True)
        ax_table = fig.add_subplot(gs[i, 1])

        test_data = athlete_data[athlete_data['Test Type'] == test_type]
        if test_data.empty:
            print(f"No data for {test_type}")
            continue

        # Identify the previous and current phases for this specific test type
        test_data_sorted = test_data.sort_values(by=['Test Number', 'Date'], ascending=[False, False])
        phase_names = test_data_sorted['Phase'].dropna().unique()[:2]  # Extract the last two valid phases

        current_phase = phase_names[0] if len(phase_names) > 0 else 'Unknown Phase'
        previous_phase = phase_names[1] if len(phase_names) > 1 else 'Unknown Phase'

        test1_vals, test2_vals = [], []
        test1_speeds, test2_speeds = [], []

        for sub_type in sub_types:
            sub_type_data = test_data[test_data['Test Sub-Type'] == sub_type]
            if sub_type_data.empty:
                test1_vals.append(None)
                test2_vals.append(None)
                test1_speeds.append(None)
                test2_speeds.append(None)
                continue

            valid_test_numbers = sub_type_data['Test Number'].sort_values(ascending=False).unique()
            most_recent_test_number = valid_test_numbers[0]
            most_recent_test = sub_type_data[sub_type_data['Test Number'] == most_recent_test_number]

            previous_test_number = valid_test_numbers[1] if len(valid_test_numbers) > 1 else None
            previous_test = sub_type_data[sub_type_data['Test Number'] == previous_test_number] if previous_test_number else None

            test2_vals.append(most_recent_test['Weight'].values[0] if not most_recent_test.empty else None)
            test2_speeds.append(most_recent_test['Speed'].values[0] if not most_recent_test.empty else None)
            test1_vals.append(previous_test['Weight'].values[0] if previous_test is not None and not previous_test.empty else None)
            test1_speeds.append(previous_test['Speed'].values[0] if previous_test is not None and not previous_test.empty else None)

        radar_test1_vals = [0 if v is None else v for v in test1_vals]
        radar_test2_vals = [0 if v is None else v for v in test2_vals]
        radar_test1_vals += [radar_test1_vals[0]]
        radar_test2_vals += [radar_test2_vals[0]]
        angles = np.linspace(0, 2 * np.pi, len(sub_types), endpoint=False).tolist()
        angles += angles[:1]

        ax.set_theta_offset(np.pi / 2)
        ax.set_theta_direction(-1)
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(sub_types, fontsize=6, ha='center')

        if any(test1_vals):
            ax.plot(angles, radar_test1_vals, linewidth=1, linestyle='dotted', color='b', label='Test 1')
            ax.fill(angles, radar_test1_vals, 'b', alpha=0.25)

        ax.plot(angles, radar_test2_vals, linewidth=2, linestyle='solid', color='r', label='Test 2')
        ax.fill(angles, radar_test2_vals, 'r', alpha=0.25)

        max_val = max(max(radar_test1_vals), max(radar_test2_vals))
        max_y_value = max(90, np.ceil(max_val / 45) * 45)
        y_ticks = np.arange(0, max_y_value + 45, 45)
        ax.set_yticks(y_ticks)
        ax.set_yticklabels([f'{int(tick)}' for tick in y_ticks], fontsize=6)
        ax.yaxis.grid(True)

        ax.set_title(test_type, size=14, fontweight='bold')

        weight_diff = [t2 - t1 if t1 is not None and t2 is not None else None for t1, t2 in zip(test1_vals, test2_vals)]
        speed_diff = [s2 - s1 if s1 is not None and s2 is not None else None for s1, s2 in zip(test1_speeds, test2_speeds)]

        table_data = []
        for sub_type, w_diff, sp_diff in zip(sub_types, weight_diff, speed_diff):
            if w_diff is None:
                table_data.append([sub_type, 'N/A', 'N/A', 'black', 'black'])
            else:
                weight_color = 'green' if w_diff > 0 else ('red' if w_diff < 0 else 'black')
                speed_color = 'green' if sp_diff > 0 else ('red' if sp_diff < 0 else 'black')
                table_data.append([sub_type, f'{w_diff:+}', f'{sp_diff:+.2f}', weight_color, speed_color])

        col_labels = [f'{previous_phase} → {current_phase}', 'Weight Gains', 'Speed Gains']
        ax_table.axis('off')
        table = ax_table.table(cellText=[[row[0], row[1], row[2]] for row in table_data],
                               colLabels=col_labels,
                               loc='center',
                               cellLoc='center',
                               colColours=[mcolors.CSS4_COLORS['lightgray']] * 3)

        table.scale(1.2, 1.5)  # **Increase cell size**
        
        for (i, j), cell in table.get_celld().items():
            if i == 0:
                cell.set_text_props(weight='bold')

        for i, row in enumerate(table_data):
            table[(i + 1, 1)].set_text_props(color=row[3])
            table[(i + 1, 2)].set_text_props(color=row[4])

    fig.suptitle("Velocity Based Training (VBT)", fontsize=20, x=0.53, y=0.95, va='center', fontweight='bold')
    plt.figlegend(handles=legend_elements, loc='lower center', ncol=2, fontsize=10)

    pdf.savefig(fig)
    plt.close(fig)

def generate_vbt_progress_graphs(df, athlete_id, pdf):

    athlete_data = df[(df['ID'] == athlete_id) & (df['Workout Type'] == 'Strength')]
    if athlete_data.empty:
        print(f"No VBT data found for athlete ID {athlete_id}.")
        return

    test_types = ['Back Squat', 'Bench Press', 'Deadlift']
    sub_types = ['Starting Strength', 'Strength-Speed', 'Accelerative Strength', 'Absolute Strength']

    fig, axs = plt.subplots(len(test_types), len(sub_types), figsize=(14, 12), sharex='col')
    fig.subplots_adjust(hspace=0.4, wspace=0.3)

    athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"

    # Iterate over Test Types and Sub-Types
    for i, test_type in enumerate(test_types):
        # Filter Test Numbers dynamically for this Test Type
        test_type_data = athlete_data[athlete_data['Test Type'] == test_type]
        unique_test_numbers = test_type_data['Test Number'].unique()
        unique_test_numbers.sort()  # Ensure sorted order

        if not test_type_data.empty:
            max_weight = test_type_data['Weight'].max()
            weight_limit = max(90, np.ceil(max_weight * 1.15))  # **Increased dynamically**
        else:
            weight_limit = 315

        if not test_type_data.empty and 'Speed' in test_type_data.columns:
            max_speed = test_type_data['Speed'].max()
            speed_limit = max(1.0, max_speed * 1.2)  # **Scale speed Y-limit dynamically**
        else:
            speed_limit = 1.3  # Default cap if no speed data is available

        for j, sub_type in enumerate(sub_types):
            ax = axs[i, j]
            ax2 = ax.twinx()  # Create a twin Y-axis for speed

            # Filter data for the specific Test Type and Sub-Type
            sub_type_data = athlete_data[(athlete_data['Test Type'] == test_type) & 
                                         (athlete_data['Test Sub-Type'] == sub_type)]
            if sub_type_data.empty:
                ax.text(0.5, 0.5, "No Data", ha='center', va='center', fontsize=10, color='gray')
                ax.axis('off')
                continue

            # Sort data by Test Number for consistent plotting
            sub_type_data = sub_type_data.sort_values(by='Test Number')

            # Map Test Numbers to an index for uniformity within the Test Type
            test_number_mapping = {test_num: idx + 1 for idx, test_num in enumerate(unique_test_numbers)}
            sub_type_data['Mapped Test Number'] = sub_type_data['Test Number'].map(test_number_mapping)

            # Plot Weight and Speed over Mapped Test Numbers
            ax.plot(sub_type_data['Mapped Test Number'], sub_type_data['Weight'], marker='o', label='Weight (lbs)', color='blue', linewidth=2)
            ax2.plot(sub_type_data['Mapped Test Number'], sub_type_data['Speed'], marker='s', label='Speed (m/s)', color='orange', linewidth=2)

            # Annotate values for clarity
            for k in range(len(sub_type_data)):
                mapped_test_num = sub_type_data['Mapped Test Number'].iloc[k]
                weight = sub_type_data['Weight'].iloc[k]
                speed = sub_type_data['Speed'].iloc[k]
                phase = sub_type_data['Phase'].iloc[k] if 'Phase' in sub_type_data.columns else 'N/A'
                ax.text(mapped_test_num, weight, f"{weight:.1f}", ha='center', va='bottom', fontsize=8, color='blue')
                ax2.text(mapped_test_num, speed - 0.05, f"{speed:.2f}", ha='center', va='top', fontsize=8, color='orange')
                ax.text(mapped_test_num, -0.1 * weight_limit, phase, ha='center', va='top', fontsize=8, rotation=45, color='black')

            # **Set dynamic Y-axis limits for centering**
            ax.set_ylim(0, weight_limit)
            ax2.set_ylim(0, speed_limit)

            # Add labels only for specific graphs
            if sub_type == 'Starting Strength':
                ax.set_ylabel("Weight (lbs)", fontsize=9, color='blue')
            if sub_type == 'Absolute Strength':
                ax2.set_ylabel("Speed (m/s)", fontsize=9, color='orange')

            # Aesthetics
            ax.set_title(f"{test_type} - {sub_type}", fontsize=10, fontweight='bold')
            ax.grid(alpha=0.3)

            # Customize ticks for readability
            ax.tick_params(axis='x', labelsize=8, length=0)  # Remove default tick labels
            ax.tick_params(axis='y', labelsize=8, colors='blue')
            ax2.tick_params(axis='y', labelsize=8, colors='orange')

            # Update x-tick labels to reflect mapped test numbers
            ax.set_xticks(range(1, len(unique_test_numbers) + 1))
            ax.set_xticklabels([str(test_num) for test_num in unique_test_numbers], fontsize=8)

    # Add overall title
    fig.suptitle(f"{athlete_name} - VBT Historical Progress", fontsize=16, fontweight='bold', y=0.98)

    # Save the plot to the PDF
    pdf.savefig(fig)
    plt.close(fig)

# Function to save jump testing plots
def plot_jump_testing(df, athlete_id, pdf):
   
    # Retrieve athlete's info
    athlete_info = df[df['ID'] == athlete_id][['First Name', 'Last Name', 'Level']].iloc[0]
    athlete_name = f"{athlete_info['First Name']} {athlete_info['Last Name']}"
    athlete_level = athlete_info['Level']

    # Filter for Jump Tests and separate Broad and Vertical data
    jump_data = df[df['Test Type'] == 'Jump']
    broad_data = jump_data[jump_data['Test Sub-Type'] == 'Broad'][['ID', 'Test Number', 'Distance']]
    vertical_data = jump_data[jump_data['Test Sub-Type'] == 'Vertical'][['ID', 'Test Number', 'Vert']]

    # Ensure data copies before modifying
    broad_data = broad_data.copy()
    vertical_data = vertical_data.copy()

    # Convert to numeric and drop NaNs
    broad_data['Distance'] = pd.to_numeric(broad_data['Distance'], errors='coerce')
    vertical_data['Vert'] = pd.to_numeric(vertical_data['Vert'], errors='coerce')

    broad_data = broad_data.dropna(subset=['Distance'])
    vertical_data = vertical_data.dropna(subset=['Vert'])

    # Merge broad and vertical data on 'ID' and 'Test Number'
    merged_jump_data = pd.merge(broad_data, vertical_data, on=['ID', 'Test Number'], how='inner')

    # Get the athlete's jump test data
    athlete_jump_data = merged_jump_data[merged_jump_data['ID'] == athlete_id]
    if athlete_jump_data.empty:
        print("No jump test data found for the specified athlete.")
        return

    # Remove the athlete's data from facility-wide data to avoid overlap
    facility_data = merged_jump_data[merged_jump_data['ID'] != athlete_id]

    # Filter facility data by Level
    level_filtered_data = facility_data.merge(df[['ID', 'Level']], on='ID')
    level_filtered_data = level_filtered_data[level_filtered_data['Level'] == athlete_level]

    # Identify the athlete's most recent and other test data points
    highest_test_number = athlete_jump_data['Test Number'].max()
    specified_test = athlete_jump_data[athlete_jump_data['Test Number'] == highest_test_number]
    other_athlete_tests = athlete_jump_data[athlete_jump_data['Test Number'] != highest_test_number]

    # Create the plot
    fig, ax = plt.subplots(figsize=(14, 8))

    # Plot all facility-wide jump tests (excluding the specified athlete)
    ax.scatter(facility_data['Distance'], facility_data['Vert'], alpha=0.6, label='All Athletes', color='lightgray', marker='o')

    # Plot level-filtered data points (blue)
    ax.scatter(level_filtered_data['Distance'], level_filtered_data['Vert'], alpha=0.6, label=f"{athlete_level} Group", color='blue', marker='o')

    # Plot athlete's other tests (green)
    ax.scatter(other_athlete_tests['Distance'], other_athlete_tests['Vert'], alpha=0.8, color='green', label=f"Other Tests by {athlete_name}", marker='^')

    # Plot athlete's most recent test (red)
    ax.scatter(specified_test['Distance'], specified_test['Vert'], alpha=1.0, color='red', label=f"Most Recent Test by {athlete_name}", marker='D')

    # **Add Line of Best Fit for the Level Group (Blue Data)**
    if not level_filtered_data.empty:
        x = level_filtered_data['Distance']
        y = level_filtered_data['Vert']

        # Calculate linear regression coefficients (slope and intercept)
        slope, intercept = np.polyfit(x, y, 1)

        # Generate fitted y-values for the trend line
        x_range = np.linspace(x.min(), x.max(), 100)
        y_fit = slope * x_range + intercept

        # Plot the line of best fit
        ax.plot(x_range, y_fit, color='blue', linestyle='--', linewidth=2, label=f"{athlete_level} Group Trend")

    # Labels and title
    ax.set_xlabel("Broad Jump Distance (ft)")
    ax.set_ylabel("Vertical Jump Vert (in)")
    ax.set_title(f"Jump Tests for {athlete_level} and Athlete", fontsize=14, fontweight='bold')

    ax.legend()

    # Save the figure to the PDF
    pdf.savefig(fig)
    plt.close(fig)

def calculate_grip_balance(df, athlete_id):
    """
    Calculate the grip balance ratio (Arm Side / Glove Side) for a given athlete.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        athlete_id (int): The ID of the athlete.

    Returns:
        float or None: Grip balance ratio, or None if data is incomplete or missing.
    """
    athlete_data = df[(df['ID'] == athlete_id) & (df['Test Type'] == 'Grip')]
    if athlete_data.empty:
        print(f"No Grip Test data found for athlete ID {athlete_id}.")
        return None

    # Filter by current level
    athlete_level = athlete_data['Level'].iloc[-1]
    peer_data = df[(df['Test Type'] == 'Grip') & (df['Level'] == athlete_level)]

    # Find the highest test number for the athlete
    latest_test_number = athlete_data['Test Number'].max()
    latest_test_data = athlete_data[athlete_data['Test Number'] == latest_test_number]

    arm_side = latest_test_data[latest_test_data['Test Sub-Type'] == 'Arm Side']['Weight'].mean()
    glove_side = latest_test_data[latest_test_data['Test Sub-Type'] == 'Glove Side']['Weight'].mean()

    if pd.isna(arm_side) or pd.isna(glove_side):
        print(f"Incomplete Grip Test data for athlete ID {athlete_id} in latest test.")
        return None

    return arm_side / glove_side

def calculate_shoulder_balance(df, athlete_id):
    """
    Calculate the shoulder balance for a given athlete.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        athlete_id (int): The ID of the athlete.

    Returns:
        float or None: Shoulder balance, or None if data is incomplete or missing.
    """
    athlete_data = df[(df['ID'] == athlete_id) & 
                      (df['Test Type'] == 'ArmCare') & 
                      (df['Test Sub-Type'] == 'Fresh Exam')]
    if athlete_data.empty:
        print(f"No ArmCare - Fresh Exam data found for athlete ID {athlete_id}.")
        return None

    # Filter by current level
    athlete_level = athlete_data['Level'].iloc[-1]
    peer_data = df[(df['Test Type'] == 'ArmCare') & (df['Test Sub-Type'] == 'Fresh Exam') & (df['Level'] == athlete_level)]

    latest_test_number = athlete_data['Test Number'].max()
    latest_test_data = athlete_data[athlete_data['Test Number'] == latest_test_number]

    shoulder_balance = latest_test_data['Shoulder Balance'].iloc[0]
    if pd.isna(shoulder_balance):
        print(f"Missing Shoulder Balance value for athlete ID {athlete_id} in latest test.")
        return None

    return shoulder_balance

def calculate_facility_grip_balance(df, level):
    """
    Calculate the facility-wide grip balance for a specific level.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        level (str): The competition level to filter by.

    Returns:
        float or None: Facility-wide grip balance, or None if data is missing.
    """
    grip_data = df[(df['Test Type'] == 'Grip') & (df['Level'] == level)]

    def calculate_ratio(group):
        arm_side = group[group['Test Sub-Type'] == 'Arm Side']['Weight'].mean()
        glove_side = group[group['Test Sub-Type'] == 'Glove Side']['Weight'].mean()

        if pd.isna(arm_side) or pd.isna(glove_side):
            return None
        return arm_side / glove_side

    facility_grip_ratios = grip_data.groupby('ID').apply(calculate_ratio).dropna()
    facility_grip_avg = facility_grip_ratios.mean()
    return facility_grip_avg

def calculate_facility_shoulder_balance(df, level):
    """
    Calculate the facility's average shoulder balance for the specified level.
    """
    shoulder_data = df[(df['Level'] == level) & df['Shoulder Balance'].notna()].copy()  # Ensure a copy

    if shoulder_data.empty:
        print(f"No facility data for Shoulder Balance at level '{level}'.")
        return 1.00  # Default to balanced if data is missing

    # Ensure numeric data and handle non-numeric entries gracefully
    shoulder_data['Shoulder Balance'] = pd.to_numeric(shoulder_data['Shoulder Balance'], errors='coerce')

    # Calculate mean after converting non-numeric to NaN and dropping them
    facility_avg_shoulder = shoulder_data['Shoulder Balance'].mean(skipna=True)

    if pd.isna(facility_avg_shoulder):
        return 1.00  # Default if no valid numeric data exists

    return facility_avg_shoulder

def calculate_jump_balance(df, athlete_id, jump_type):
    """
    Calculate the jump balance ratio for a given athlete.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        athlete_id (int): The ID of the athlete.
        jump_type (str): The type of jump ("Lateral" or "Vertical").

    Returns:
        float or None: Jump balance ratio, or None if data is incomplete or missing.
    """
    athlete_data = df[(df['ID'] == athlete_id) & (df['Test Type'] == "Jump")]

    if athlete_data.empty:
        return None

    athlete_level = athlete_data['Level'].iloc[-1]
    peer_data = df[(df['Test Type'] == 'Jump') & (df['Level'] == athlete_level)]

    block_col, load_col = ("Lateral Block Leg", "Lateral Load Leg") if jump_type == "Lateral" else ("Vertical Block Leg", "Vertical Load Leg")
    metric_col = "Distance" if jump_type == "Lateral" else "Vert"

    block_data = athlete_data[athlete_data['Test Sub-Type'] == block_col]
    load_data = athlete_data[athlete_data['Test Sub-Type'] == load_col]

    if block_data.empty or load_data.empty:
        return None

    block_leg = block_data[metric_col].iloc[-1] if not block_data[metric_col].isna().all() else None
    load_leg = load_data[metric_col].iloc[-1] if not load_data[metric_col].isna().all() else None

    if block_leg is None or load_leg is None:
        return None

    return block_leg / load_leg

def calculate_facility_jump_balance(df, jump_type, level):
    """
    Calculate the facility-wide jump balance for a specific jump type and level.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        jump_type (str): The type of jump ("Lateral" or "Vertical").
        level (str): The competition level to filter by.

    Returns:
        float or None: Facility-wide jump balance, or None if data is incomplete or missing.
    """
    block_col, load_col = ("Lateral Block Leg", "Lateral Load Leg") if jump_type == "Lateral" else ("Vertical Block Leg", "Vertical Load Leg")
    metric_col = "Distance" if jump_type == "Lateral" else "Vert"

    facility_data = df[(df['Test Sub-Type'] == block_col) & (df['Level'] == level)]
    facility_block = facility_data[metric_col].mean()

    facility_data = df[(df['Test Sub-Type'] == load_col) & (df['Level'] == level)]
    facility_load = facility_data[metric_col].mean()

    if facility_block > 0 and facility_load > 0:
        return facility_block / facility_load
    return None

def plot_balance_graph_with_labels(df, athlete_id, pdf, verbose=False):
    
    # Get athlete's information
    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        print(f"No data found for athlete ID {athlete_id}.")
        return

    athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"
    athlete_level = athlete_data['Level'].iloc[-1]

    # Calculate athlete-specific values
    athlete_grip_balance = calculate_grip_balance(df, athlete_id)
    athlete_shoulder_balance = calculate_shoulder_balance(df, athlete_id)
    athlete_lateral_jump_balance = calculate_jump_balance(df, athlete_id, "Lateral")
    athlete_vertical_jump_balance = calculate_jump_balance(df, athlete_id, "Vertical")

    # Calculate facility averages for the same level
    facility_grip_avg = calculate_facility_grip_balance(df, athlete_level)
    facility_shoulder_avg = calculate_facility_shoulder_balance(df, athlete_level)
    facility_lateral_jump_avg = calculate_facility_jump_balance(df, "Lateral", athlete_level)
    facility_vertical_jump_avg = calculate_facility_jump_balance(df, "Vertical", athlete_level)

    # Define ranges and categories
    x_range = (0.5, 1.5)
    regions = {
        "Warning": [(0.5, 0.69), (1.31, 1.5)],
        "Caution": [(0.7, 0.84), (1.16, 1.3)],
        "Good": [(0.85, 0.99), (1.01, 1.15)],
        "Optimal": [(1.0, 1.0)]
    }

    # Define metrics and their values
    metrics = {
        "Shoulder Balance": {"value": athlete_shoulder_balance, "facility_avg": facility_shoulder_avg},
        "Grip Balance": {"value": athlete_grip_balance, "facility_avg": facility_grip_avg},
        "Lateral Jump Balance": {"value": athlete_lateral_jump_balance, "facility_avg": facility_lateral_jump_avg},
        "Vertical Jump Balance": {"value": athlete_vertical_jump_balance, "facility_avg": facility_vertical_jump_avg},
    }

    # Function to determine color based on balance value
    def get_color(value):
        if value is None:
            return "gray"
        if 0.5 <= value <= 0.69 or 1.31 <= value <= 1.5:
            return "red"  # Warning
        if 0.7 <= value <= 0.84 or 1.16 <= value <= 1.3:
            return "yellow"  # Caution
        if 0.85 <= value <= 0.949 or 1.051 <= value <= 1.15:
            return "green"  # Good
        if 0.95 <= value <= 1.05:
            return "blue"  # Optimal
        return "gray"

    # Set up the plot
    fig, ax = plt.subplots(figsize=(14, 5))

    # Add gradient background
    gradient = np.linspace(0.5, 1.5, 1000).reshape(1, -1)
    gradient_colors = np.abs(gradient - 1)
    ax.imshow(
        gradient_colors,
        aspect="auto",
        extent=[0.5, 1.5, 0.46, 0.54],
        cmap="coolwarm_r",
        alpha=0.8
    )

    # Draw labels for balance regions
    for region, ranges in regions.items():
        for start, end in ranges:
            ax.text(
                (start + end) / 2, 0.5,
                region,
                ha="center",
                va="center",
                fontsize=10 if region == "Optimal" else 14,
                alpha=0.2,
                fontweight="bold",
                color="black",
                transform=ax.transData
            )

    # Plot data points for each metric
    y_positions = np.linspace(0.48, 0.52, len(metrics) * 2)
    label_offsets = np.linspace(0.55, 0.59, len(metrics))
    label_h_positions = np.linspace(0.6, 1.4, len(metrics))

    for i, (metric, details) in enumerate(metrics.items()):
        value = details["value"]
        facility_avg = details["facility_avg"]

        if value is None:
            if verbose:
                print(f"Skipping {metric}: Athlete data missing.")
            continue

        color = get_color(value)
        athlete_y = y_positions[i * 2]
        label_y = label_offsets[i % len(label_offsets)] - 0.03
        label_x = label_h_positions[i % len(label_h_positions)]

        # Plot athlete data
        ax.plot(
            [value],
            [athlete_y],
            marker='o',
            color=color,
            markersize=10
        )

        # Plot facility average if available
        if facility_avg is not None:
            ax.plot(
                [facility_avg],
                [athlete_y],
                marker='x',
                color="lightgray",
                markersize=8,
                alpha=0.6
            )
            ax.plot(
                [value, facility_avg],
                [athlete_y, athlete_y],
                linestyle="dotted",
                color="gray",
                linewidth=1.5
            )

        # Add a line connecting the annotation to the data point
        ax.plot(
            [value, label_x],
            [athlete_y, label_y],
            linestyle="solid",
            color="lightblue",
            linewidth=1.5
        )

        # Annotate with metric name and balance status
        if metric == "Shoulder Balance":
            dominance = (
        "ER Dominant" if value > 1.15 else
        "IR Dominant" if value < 0.85 else
        "Balanced"
    )
        elif metric == "Grip Balance":
            dominance = (
        "Arm Side Dominant" if value > 1.05 else
        "Glove Side Dominant" if value < 0.95 else
        "Balanced"
    )
        elif metric == "Lateral Jump Balance":
            dominance = (
        "Block Leg Dominant" if value > 1.05 else
        "Load Leg Dominant" if value < 0.95 else
        "Balanced"
    )
        elif metric == "Vertical Jump Balance":
            dominance = (
        "Block Leg Dominant" if value > 1.05 else
        "Load Leg Dominant" if value < 0.95 else
        "Balanced"
    )
        else:
            dominance = "Balanced"  # Default fallback

        ax.annotate(
            f"{metric}\n{value:.2f}\n{dominance}",
            xy=(label_x, label_y),
            ha="center",
            fontsize=9,
            color="black",
            bbox=dict(boxstyle="round,pad=0.3", edgecolor="gray", facecolor="white", alpha=0.8),
        )

    # Finalize plot aesthetics
    ax.set_xlim(x_range)
    ax.set_ylim(0.4, 0.6)
    ax.set_xticks(np.linspace(0.5, 1.5, 11))
    ax.set_xticklabels([f"{tick:.2f}" for tick in np.linspace(0.5, 1.5, 11)], fontsize=10)
    ax.set_yticks([])
    ax.set_xlabel("Balance Ratio", fontsize=12, fontweight="bold")
    ax.set_title(f"{athlete_name} - Imbalance Visualization\nLevel: {athlete_level}", fontsize=14, fontweight="bold")
    ax.grid(axis="x", linestyle="--", linewidth=0.5, alpha=0.7)

    # Save the plot to the PDF
    pdf.savefig(fig)
    plt.close(fig)

# Function to save velocity with arm score plots
def plot_velocity_with_arm_score(df, athlete_id, pdf):
    athlete_data = df[df['ID'] == athlete_id]
    armcare_data = athlete_data.dropna(subset=['Arm Score'])

    # Extract all relevant pitching test types, including Mound Velo & Catchers Velo
    pitching_data = athlete_data[
        (athlete_data['Workout Type'] == 'Pitching') & 
        (athlete_data['Test Sub-Type'] == '5oz')
    ]
    trackman_data = athlete_data[
        (athlete_data['Test Type'] == 'Trackman Bullpen') & 
        (athlete_data['Test Sub-Type'] == 'Fastball')
    ]

    # Parse dates and sort
    for data in [armcare_data, pitching_data, trackman_data]:
        if not data.empty:
            data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
            data.dropna(subset=['Date'], inplace=True)
            data.sort_values('Date', inplace=True)

    # Ensure valid armcare data
    if not armcare_data.empty:
        armcare_data['Arm Score'] = pd.to_numeric(armcare_data['Arm Score'], errors='coerce')
        armcare_data.dropna(subset=['Arm Score'], inplace=True)

    # **Updated test types to include "Mound Velo" & "Catchers Velo"**
    test_types = ['Roll Ins', 'Double Plays', 'Turn and Burns', 'Pulldowns', 
                  'Trackman Bullpen', 'Mound Velo', 'Catchers Velo']
    colors = ['red', 'green', 'blue', 'purple', 'orange', 'darkcyan', 'brown']
    velocity_changes = []

    for i, test_type in enumerate(test_types):
        if test_type == 'Trackman Bullpen':
            type_data = trackman_data
        else:
            type_data = pitching_data[pitching_data['Test Type'] == test_type]

        if len(type_data) >= 2:
            most_recent_velo = type_data.iloc[-1]['Max Velo']
            previous_velo = type_data.iloc[-2]['Max Velo']
            all_time_velo = type_data['Max Velo'].max()
            today_change = most_recent_velo - previous_velo
            today_pct_change = (today_change / previous_velo) * 100 if previous_velo != 0 else 0
            all_time_change = most_recent_velo - type_data.iloc[0]['Max Velo']
            all_time_pct_change = (all_time_change / type_data.iloc[0]['Max Velo']) * 100 if type_data.iloc[0]['Max Velo'] != 0 else 0
            is_pr = " PR" if most_recent_velo == all_time_velo else ""

            today_color = 'black' if today_change == "N/A" else ('green' if today_change > 0 else 'red')
            all_time_color = 'black' if all_time_change == "N/A" else ('green' if all_time_change > 0 else 'red')

            velocity_changes.append([test_type, f"{most_recent_velo:.2f} MPH   {is_pr}",
                                     f"{today_change:+.2f} ({today_pct_change:+.1f}%)", today_color,
                                     f"{all_time_change:+.2f} ({all_time_pct_change:+.1f}%)", all_time_color, colors[i]])
        elif len(type_data) == 1:
            most_recent_velo = type_data.iloc[-1]['Max Velo']
            is_pr = " PR" if most_recent_velo == type_data['Max Velo'].max() else ""
            velocity_changes.append([test_type, f"{most_recent_velo:.2f} MPH   {is_pr}", "N/A", "black", "N/A", "black", colors[i]])
        else:
            velocity_changes.append([test_type, "N/A", "N/A", "black", "N/A", "black", colors[i]])

    fig = plt.figure(figsize=(14, 12))
    gs = fig.add_gridspec(nrows=3, ncols=1, height_ratios=[0.3, 1, 4])

    fig.suptitle(f"BEA Throwing Drill Velocity and Arm Score Over Time", fontsize=16, fontweight='bold', y=0.97)
    ax_table = fig.add_subplot(gs[1])
    ax_table.axis('off')

    col_labels = ['BEA Drill', 'Velo', 'Today', 'All Time']
    table_data = []
    for row in velocity_changes:
        formatted_velo = row[1].replace("PR", r"$\bf{PR}$")  # Bold PR
        table_data.append([row[0], formatted_velo, row[2], row[4]])

    table = ax_table.table(cellText=table_data, colLabels=col_labels, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)

    for (i, j), cell in table.get_celld().items():
        if i == 0:
            cell.set_text_props(weight='bold')
            cell.set_facecolor('#D3D3D3')
        elif j == 0:
            cell.set_text_props(color=velocity_changes[i - 1][6])  # Throw type color
        elif j == 2:
            cell.set_text_props(color=velocity_changes[i - 1][3])  # Today color
        elif j == 3:
            cell.set_text_props(color=velocity_changes[i - 1][5])  # All time color

    ax1 = fig.add_subplot(gs[2])
    handles = []
    all_velo_values = []

    for test_type, color, marker in zip(test_types, colors, ['o', 'x', 's', 'D', '^', 'P', '*']):
        if test_type == 'Trackman Bullpen':
            test_data = trackman_data
        else:
            test_data = pitching_data[pitching_data['Test Type'] == test_type]

        if not test_data.empty:
            line_max, = ax1.plot(test_data['Date'], test_data['Max Velo'], color=color, marker=marker, label=f'{test_type} - Max Velo')
            handles.append((line_max, f'{test_type} - Max Velo'))
            all_velo_values.extend(test_data['Max Velo'])

             # **Annotate the last velocity score**
            last_entry = test_data.iloc[-1]
            ax1.text(last_entry['Date'], last_entry['Max Velo'] + 1, f"{last_entry['Max Velo']:.1f} MPH",
                    color=color, ha='center', fontsize=9, fontweight='bold')

    # Ensure min_velo and max_velo are always assigned, even if no throwing data exists
    if all_velo_values:
        min_velo = max(50, min(all_velo_values) * 0.9)  # Ensures the lowest plotted value is at least 50
        max_velo = max(all_velo_values) * 1.1  # Gives spacing at the top
    else:
        min_velo = 50  # Default minimum if no throwing data
        max_velo = 100  # Default maximum if no throwing data


    if not armcare_data.empty:
        num_data_points = len(armcare_data)
        bar_width = max(0.8, min(10 / num_data_points, 0.2))  # Dynamically scale but cap max width
        bars = ax1.bar(armcare_data['Date'], armcare_data['Arm Score'], width=bar_width, color='skyblue', alpha=0.7)
        handles.append((bars[0], 'Arm Score'))

        max_arm_score = armcare_data['Arm Score'].max()
        min_arm_score = armcare_data['Arm Score'].min() if len(armcare_data) > 1 else 50  # Prevents bars from plotting too low
        arm_score_limit = max(100, max_arm_score * 1.1)

        ax1.set_ylim(min(min_velo, min_arm_score * 0.9), max(max_velo, arm_score_limit))  # Unifies Y-axis scaling

            # **Highlight max arm score in gold**
        for bar, score in zip(bars, armcare_data['Arm Score']):
            if score == max_arm_score:
                bar.set_color('gold')  # Change to gold
                ax1.text(bar.get_x() + bar.get_width() / 2, score + 0.5, f"{score:.1f}", 
                        color='black', ha='center', fontsize=10, fontweight='bold')

    ax1.set_ylabel("Velocity (MPH) and Arm Score")
    ax1.yaxis.grid(True, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)

    if not armcare_data.empty:
        ax1.set_xticks(armcare_data['Date'])
        ax1.set_xticklabels(armcare_data['Date'].dt.strftime('%m/%d'), rotation=45, ha='right', fontsize=8)

    plt.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)

def calculate_facility_averages(df, metrics, levels_column="Level"):
    """
    Calculate the 50th percentile (median) for specified metrics at each Level.

    Parameters:
    - df: Pandas DataFrame containing test data.
    - metrics: List of metrics to calculate facility averages for (e.g., ["Max Bat Speed", "Average EV"]).
    - levels_column: Column name representing competition levels (default: "Level").

    Returns:
    - facility_averages: Dictionary containing median values for each metric grouped by level.
    """
    facility_averages = {}

    for metric in metrics:
        # Convert the metric column to numeric, coercing errors to NaN
        df[metric] = pd.to_numeric(df[metric], errors="coerce")

        # Group by level and calculate the median for the current metric
        facility_averages[metric] = (
            df.groupby(levels_column)[metric].median().dropna().to_dict()
        )

    return facility_averages

def generate_hitting_pr_board(df, athlete_id, pdf):
    """
    Generate a scoreboard-style PR Board for the athlete's hitting assessment.

    Parameters:
    - df: Pandas DataFrame containing test data.
    - athlete_id: Athlete's ID (string or integer).
    - pdf: PdfPages object for saving pages.
    """
    # Filter athlete-specific data
    athlete_data = df[df['ID'] == athlete_id]

    # Check for "Hitting" in Workout Type
    hitting_data = athlete_data[athlete_data['Workout Type'] == 'Hitting']
    if hitting_data.empty:
        print(f"Skipping Hitting PR Board for Athlete ID {athlete_id}: No hitting data available.")
        return  # Skip if no hitting data is available

    # Convert relevant metric columns to numeric (force invalid values to NaN)
    hitting_metrics = ["Max Bat Speed", "Max EV", "Max Distance", "Peak Hand Speed", "Average EV"]
    athlete_data.loc[:, hitting_metrics] = athlete_data[hitting_metrics].apply(pd.to_numeric, errors="coerce")



    # Proceed with PR board generation
    athlete_data.loc[:, "Date"] = pd.to_datetime(athlete_data["Date"], errors="coerce")
    athlete_data = athlete_data.dropna(subset=["Date"]).sort_values(by="Date")

    # PR Calculation (handling NaN safely)
    prs = {}
    for metric in hitting_metrics:
        valid_data = athlete_data.dropna(subset=[metric])  # Ensure only valid numerical data is considered
        max_row = valid_data.loc[valid_data[metric].idxmax()] if not valid_data.empty else None
        prs[metric] = {
            "value": max_row[metric] if max_row is not None else None,
            "date": max_row["Date"].strftime("%m/%d/%Y") if max_row is not None else "N/A",
            "phase": max_row["Phase"] if max_row is not None else "N/A",
            "test_sub_type": max_row["Test Sub-Type"] if max_row is not None else "N/A",
        }

    # Load the custom font
    font_path = r"C:\Users\benoi\AppData\Local\Microsoft\Windows\Fonts\SfDigitalReadoutHeavyOblique-GKRA.ttf"
    custom_font = font_manager.FontProperties(fname=font_path)

    # Create the page layout
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.axis("off")

    # PR Board Header
    header = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]} Hitting Assessment PR Board"
    ax.text(0.5, 0.95, header, fontsize=20, fontweight="bold", ha="center", va="center")

    # Display PR Metrics
    metrics_display_order = ["Max Bat Speed", "Max EV", "Max Distance", "Peak Hand Speed", "Average EV"]
    positions = [
        (0.15, 0.65),  # Max Bat Speed
        (0.5, 0.65),  # Max EV
        (0.85, 0.65),  # Max Distance
        (0.35, 0.3),  # Peak Hand Speed
        (0.65, 0.3),  # Average EV
    ]
    font_sizes = [84, 84, 84, 48, 48]  # Larger fonts for primary metrics

    for metric, pos, font_size in zip(metrics_display_order, positions, font_sizes):
        pr = prs[metric]
        value = pr["value"]
        value_text = f"{value:.2f}" if value is not None else "N/A"
        metric_text = f"{metric}"
        details = f"{pr['date']} | {pr['phase']} | {pr['test_sub_type']}"

        # Add metric name
        ax.text(pos[0], pos[1] + 0.12, metric_text, fontsize=16, ha="center", va="center", fontweight="bold")

        # Add value in digital scoreboard font
        ax.text(
            pos[0],
            pos[1],
            value_text,
            fontsize=font_size,
            fontproperties=custom_font,
            color="darkred",
            ha="center",
            va="center",
        )

        # Add smaller details below
        detail_offset = -0.1 if font_size == 84 else -0.06
        ax.text(pos[0], pos[1] + detail_offset, details, fontsize=8, ha="center", va="center", wrap=True)

    # Save the page to PDF
    pdf.savefig(fig)
    plt.close(fig)

def plot_hitting_metrics_combined(df, athlete_id, metrics, facility_averages, pdf):
    """
    Plot historical progress of hitting metrics for an athlete on one page.

    Parameters:
    - df: Pandas DataFrame containing test data.
    - athlete_id: Athlete's ID (string or integer).
    - metrics: List of metrics to plot (e.g., ["Max Bat Speed", "Average EV"]).
    - facility_averages: Dictionary of facility average values grouped by level.
    - pdf: PdfPages object for saving plots.
    """
    # Filter athlete-specific data
    athlete_data = df[df['ID'] == athlete_id]

    # Check for "Hitting" in Workout Type
    hitting_data = athlete_data[athlete_data['Workout Type'] == 'Hitting']
    if hitting_data.empty:
        print(f"Skipping Hitting Metrics Plot for Athlete ID {athlete_id}: No hitting data available.")
        return  # Skip if no hitting data is available

    # Convert date column
    athlete_data = athlete_data.copy()
    athlete_data["Date"] = pd.to_datetime(athlete_data["Date"], errors="coerce")
    athlete_data = athlete_data.dropna(subset=["Date"]).sort_values(by="Date")

    # Test sub-types and their colors
    test_sub_types = athlete_data["Test Sub-Type"].dropna().unique()
    sub_type_colors = {sub: plt.cm.tab10(i) for i, sub in enumerate(test_sub_types)}

    # Create a single figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    axes = axes.flatten()

    for i, metric in enumerate(metrics):
        ax = axes[i]

        # Plot athlete data by test sub-type
        for sub_type, color in sub_type_colors.items():
            sub_data = athlete_data[athlete_data["Test Sub-Type"] == sub_type]
            if not sub_data.empty and metric in sub_data.columns:
                valid_data = sub_data.dropna(subset=[metric])
                if not valid_data.empty:
                    ax.plot(
                        valid_data["Date"],
                        valid_data[metric],
                        marker="o",
                        linestyle="-",
                        label=sub_type,  # Ensure the legend tracks the sub-type
                        color=color,
                    )
                    # Annotate each data point
                    for _, row in valid_data.iterrows():
                        ax.text(
                            row["Date"],
                            row[metric],
                            f"{row[metric]:.1f}",
                            fontsize=8,
                            ha="center",
                            va="bottom",
                        )

        # Plot facility average
        athlete_level = athlete_data["Level"].iloc[-1]
        if metric in facility_averages and athlete_level in facility_averages[metric]:
            ax.axhline(
                facility_averages[metric][athlete_level],
                color="gray",
                linestyle="--",
                label="Facility Average",
            )

        # Customize subplot
        ax.set_title(f"{metric} Progress", fontsize=14, fontweight="bold")
        ax.set_xlabel("Date", fontsize=10)
        ax.set_ylabel(metric, fontsize=10)
        ax.grid(True, linestyle="--", alpha=0.7)

        # Format x-axis to show only test dates
        unique_dates = athlete_data.dropna(subset=[metric])["Date"].drop_duplicates()
        formatted_dates = unique_dates.dt.strftime("%m/%d/%Y")
        ax.set_xticks(unique_dates)
        ax.set_xticklabels(formatted_dates, rotation=45, ha="right", fontsize=8)

    # Add legend in the top-right corner (outside the plot area)
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(
        handles,
        labels,
        title="Test Sub-Type",
        loc="upper right",
        fontsize=10,
    )

    # Adjust layout
    fig.tight_layout(rect=[0, 0, 1, 0.95])  # Leave space for the legend
    fig.suptitle(f"Hitting Metrics Progress for Athlete ID: {athlete_id}", fontsize=16, fontweight="bold")

    # Save the figure to PDF
    pdf.savefig(fig)
    plt.close(fig)

def generate_cover_page(df, athlete_id, pdf):
    # Get athlete details
    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        athlete_name = f"Athlete {athlete_id}"
        status = "N/A"
        dob = "N/A"
        level = "N/A"
        school_org = "N/A"
        grad_year = "N/A"
        position = "N/A"
        weight = "N/A"
        height = "N/A"
    else:
        athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"
        status = athlete_data['Status'].iloc[0]
        dob = athlete_data['DOB'].iloc[0] if 'DOB' in athlete_data.columns else "N/A"
        level = athlete_data['Level'].iloc[0] if 'Level' in athlete_data.columns else "N/A"
        school_org = athlete_data['School/Org'].iloc[0] if 'School/Org' in athlete_data.columns else "N/A"
        grad_year = athlete_data['Grad Year'].iloc[0] if 'Grad Year' in athlete_data.columns else "N/A"
        position = athlete_data['Position'].iloc[0] if 'Position' in athlete_data.columns else "N/A"
        
        # Get the most recent Weigh-in data
        weigh_in_data = athlete_data[athlete_data['Test Type'] == 'Weigh-in']
        if not weigh_in_data.empty:
            weight_data = weigh_in_data[weigh_in_data['Test Sub-Type'] == 'Weigh-in']
            height_data = weigh_in_data[weigh_in_data['Test Sub-Type'] == 'Height']
            weight = weight_data['Weight'].iloc[-1] if not weight_data.empty else "N/A"
            height = height_data['Height'].iloc[-1] if not height_data.empty else "N/A"
        else:
            weight = "N/A"
            height = "N/A"

    # Get the current date
    created_date = datetime.now().strftime("%B %d, %Y")

    # Create the figure for the cover page
    fig, ax = plt.subplots(figsize=(8.5, 11))  # Letter size
    ax.axis('off')  # Remove axes for a clean design

    # Title
    ax.text(
        0.5, 0.85, "Beimel Elite Athletics\nSCOPE Report",
        fontsize=30, fontweight='bold', ha='center', va='center'
    )

    # Acronym breakdown aligned vertically on the left
    acronym_text = (
        "S.trength\n"
        "C.onsistency\n"
        "O.ptimization\n"
        "P.reparation\n"
        "E.ffort"
    )
    ax.text(
        0.4, 0.65, acronym_text,
        fontsize=14, fontweight='regular', ha='left', va='center', color="gray"
    )

    # Athlete details
    details_text = (
        f"Athlete: {athlete_name}\n"
        f"Membership: {status}\n"
        f"DOB: {dob}    H/W: {height}\"/{weight}lbs \n"
        f"{school_org}  {grad_year}\n"
        f"Position: {position}\n"
        f"Created Date: {created_date}"
    )
    ax.text(
        0.5, 0.4, details_text,
        fontsize=12, fontweight='regular', ha='center', va='center'
    )

    # Footer with subtle design
    ax.text(
        0.5, 0.1, "BEA Performance Analytics",
        fontsize=10, ha='center', va='center', color="gray", alpha=0.7
    )

    # Save the figure to the PDF
    pdf.savefig(fig)
    plt.close(fig)

# Function to generate combined report
def generate_combined_report(df, athlete_id, save_directory):
    """
    Generate a combined PDF progress report for an athlete.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        athlete_id (int): Athlete ID.
        save_directory (str): Directory to save the PDF.

    Returns:
        None
    """

    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        print(f"No data found for athlete ID {athlete_id}.")
        return
    
    # File name includes athlete's name and phase
    athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"

       # Filter for the most recent phase from strength workouts
    strength_workouts = athlete_data[athlete_data['Workout Type'] == 'Strength']
    if not strength_workouts.empty:
        # Sort by Test Number and Date to ensure the latest data is selected
        recent_strength_workout = strength_workouts.sort_values(by=['Test Number', 'Date'], ascending=[False, False]).iloc[0]
        phase_name = recent_strength_workout['Phase'] if 'Phase' in recent_strength_workout else ''
    else:
        phase_name = ''

    # Get the current date
    current_date = datetime.now().strftime("%Y-%m-%d")

    file_name = f"{athlete_name.replace(' ', ' ')}, {phase_name}, {current_date}.pdf"
    pdf_path = f"{save_directory}/{file_name}"

   
    # Create the PDF filename with the new format
    #pdf_filename = f"{athlete_name}, {phase_name}, {current_date}.pdf"
    #pdf_path = os.path.join(save_directory, pdf_filename)

    selected_metrics = [
        ('Weigh-in', 'Weigh-in', 'Weight'),
        ('Weigh-in', 'Height', 'Height'),
        ('Trackman Bullpen', 'Fastball', 'Max Velo'),
        ('Trackman Bullpen', 'Fastball', 'Average Velo'),
        ('Blast Motion', 'Front Toss', 'Max Bat Speed'),
        ('Blast Motion', 'Front Toss', 'Peak Hand Speed'),
        ('HitTrax', 'Front Toss', 'Average EV'),
        ('HitTrax', 'Front Toss', 'Max EV'),
        ('HitTrax', 'Front Toss', 'Max Distance'),
        ('Roll Ins', '5oz', 'Max Velo'),
        ('Double Plays', '5oz', 'Max Velo'),
        ('Turn and Burns', '5oz', 'Max Velo'),
        ('Pulldowns', '5oz', 'Max Velo'),
        ('Jump', 'Broad', 'Distance'),
        ('Jump', 'Vertical', 'Vert'),
        ('Back Squat', 'Strength-Speed', 'Weight'),
        ('Deadlift', 'Strength-Speed', 'Weight'),
        ('Bench Press', 'Strength-Speed', 'Weight'),
        ('ArmCare', 'Fresh Exam', 'Arm Score'),
    ]
   
    custom_labels = [
        "Body Weight",
        "Height",
        "Fastball Max Velocity",
        "Fastball Average Velocity",
        "Max Bat Speed",
        "Peak Hand Speed",
        "Average Exit Velo",
        "Max Exit Velo",
        "Max Distance",
        "5oz Roll Ins",
        "5oz Double Plays",
        "5oz Turn and Burns",
        "5oz Pulldown",
        "Broad Jump Distance",
        "Vertical Jump Height",
        "Speed Back Squat",
        "Speed Deadlift",
        "Speed Bench Press",
        "ArmCare Score",
    ]

    # Metrics to plot
    metrics = ["Max Bat Speed", "Average EV", "Max EV", "Max Distance"]

    # Calculate facility averages
    facility_averages = calculate_facility_averages(df, metrics)


    with PdfPages(pdf_path) as pdf:
        # Calculate percentiles and generate radar chart
        percentiles, athlete_name, athlete_birth_year = calculate_percentiles(df, athlete_id, metric_dict)

        generate_cover_page(df, athlete_id, pdf)
       
        radar_chart_percentiles(df, percentiles, athlete_name, athlete_id, pdf, unit_dict)

        # Example usage in your progress report script
        generate_metric_table(df, athlete_id, selected_metrics, unit_dict, pdf, custom_labels)

        # Generate bar chart for improvement
        bar_chart_improvement(df, athlete_id, metric_dict, unit_dict, pdf)

        # Generate strength radar charts
        generate_strength_radar_chart(df, athlete_id, pdf)

        generate_vbt_progress_graphs(df, athlete_id, pdf)

        # Generate jump testing plots
        plot_jump_testing(df, athlete_id, pdf)

        plot_balance_graph_with_labels(df, athlete_id, pdf)

        # Generate velocity and arm score plots
        plot_velocity_with_arm_score(df, athlete_id, pdf)

        generate_hitting_pr_board(df, athlete_id, pdf)

        plot_hitting_metrics_combined(df, athlete_id, metrics, facility_averages, pdf)

    print(f"Combined progress report saved to {pdf_path}")

    # Call the combined report function

generate_combined_report(data, athlete_id=117488, save_directory='C:/Users/benoi/OneDrive/Desktop/bea/Progress Summaries/')


Skipping Hitting PR Board for Athlete ID 117488: No hitting data available.
Skipping Hitting Metrics Plot for Athlete ID 117488: No hitting data available.
Combined progress report saved to C:/Users/benoi/OneDrive/Desktop/bea/Progress Summaries//Riley Kim, Onboarding, 2025-02-07.pdf


C:\Users\benoi\AppData\Local\Temp\ipykernel_9676\71989290.py:1319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
C:\Users\benoi\AppData\Local\Temp\ipykernel_9676\71989290.py:1320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(subset=['Date'], inplace=True)
C:\Users\benoi\AppData\Local\Temp\ipykernel_9676\71989290.py:1321: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [ ]:
import matplotlib.font_manager as fm

# List all available fonts
for font in fm.findSystemFonts(fontpaths=None, fontext='ttf'):
    print(font)
